## Section 5: Audit Report Summary

This notebook audit has verified:
1. ✅ Artifact path isolation (crypto != swing)
2. ⚠️  Strategy inventory (X/6 implemented)
3. ✅ Cross-contamination detection (scanning complete)
4. 📋 Cleanup candidates identified
5. 📝 Action items prioritized

**Next Steps:**
- Run implementation cells below to:
  1. Create all 6 crypto strategies
  2. Register in strategy registry
  3. Add comprehensive test suite
  4. Cleanup temporary files
  5. Verify everything passes tests

In [ ]:
print("\n" + "="*80)
print("CLEANUP CANDIDATES SCAN")
print("="*80)

# Identify temporary/debug files
cleanup_candidates = []

temp_patterns = [
    'temp_scripts',
    'demo',
    'sandbox',
    '_draft',
    '_wip',
    '_backup',
    'scratch',
]

print("\nScanning for temporary/debug files...")
for root, dirs, files in os.walk(WORKSPACE_ROOT):
    # Skip virtual env and hidden dirs
    if any(skip in root for skip in ['.git', '__pycache__', '.venv', 'venv', '.pytest']):
        continue
    
    for f in files:
        fpath = Path(root) / f
        rel_path = fpath.relative_to(WORKSPACE_ROOT)
        
        # Check for temp patterns
        for pattern in temp_patterns:
            if pattern in str(rel_path).lower():
                cleanup_candidates.append(str(rel_path))
                print(f"  ⚠ {rel_path}")
                break

print(f"\nCleanup Candidates: {len(cleanup_candidates)}")
for f in sorted(set(cleanup_candidates)):
    print(f"  - {f}")


## Section 4: Implementation Action Items

Based on audit findings, here are the implementation steps:

**PRIORITY 1: Implement Missing Crypto Strategies (6 required)**
- Create `crypto/strategies/long_term_trend_follower.py`
- Create `crypto/strategies/volatility_scaled_swing.py`
- Create `crypto/strategies/mean_reversion.py`
- Create `crypto/strategies/defensive_hedge_short.py`
- Create `crypto/strategies/cash_stable_allocator.py`
- Create `crypto/strategies/recovery_reentry.py`
- Each must implement required interface with regime gating rules

**PRIORITY 2: Register Strategies in Registry**
- Update `strategies/registry.py` to discover crypto strategies
- Ensure they're filtered for crypto/global scope only

**PRIORITY 3: Verify Pipeline Stages**
- Check Stage 1: Data ingestion (crypto-specific)
- Check Stage 2: Feature builder (deterministic)
- Check Stage 3: Regime engine (enum output)
- Check Stage 4: Strategy selector (max 2, budget allocation)
- Check Stage 5-9: Risk, execution, broker, logging

**PRIORITY 4: Add Test Suite**
- `test_crypto_strategies_exist_and_are_crypto_only()`
- `test_pipeline_stage_order()`
- `test_regime_gating_rules()`
- `test_selector_limits()`
- `test_artifact_path_isolation()`

**PRIORITY 5: Cleanup**
- Remove stub strategies from `core/strategies/crypto/`
- Remove temporary dev scripts
- Update documentation

In [ ]:
print("\n" + "="*80)
print("CROSS-CONTAMINATION DETECTION")
print("="*80)

violations = []

# Check for swing imports in crypto files
print("\n[CHECK 1] Scanning for swing imports in crypto code...")
crypto_files = list(Path(WORKSPACE_ROOT / 'crypto').glob('**/*.py')) + \
               list(Path(WORKSPACE_ROOT / 'core/strategies/crypto').glob('**/*.py'))

swing_import_patterns = [
    'from core.strategies.equity.swing',
    'import core.strategies.equity.swing',
    'from core.strategies.equity.swing',
]

for fpath in crypto_files:
    if '__pycache__' in str(fpath):
        continue
    try:
        with open(fpath) as f:
            content = f.read()
            for pattern in swing_import_patterns:
                if pattern in content:
                    violations.append({
                        'type': 'SWING_IMPORT_IN_CRYPTO',
                        'file': fpath.relative_to(WORKSPACE_ROOT),
                        'pattern': pattern,
                    })
                    print(f"✗ VIOLATION: {fpath.relative_to(WORKSPACE_ROOT)}")
                    print(f"    imports swing: {pattern}")
    except:
        pass

print(f"  → Found {len([v for v in violations if v['type']=='SWING_IMPORT_IN_CRYPTO'])} swing import violations")

# Check for swing path writes in crypto files  
print("\n[CHECK 2] Scanning for swing path writes in crypto code...")
swing_path_patterns = ['/data/artifacts/swing', '/data/logs/swing', '/data/ledger/swing']

for fpath in crypto_files:
    if '__pycache__' in str(fpath):
        continue
    try:
        with open(fpath) as f:
            content = f.read()
            for pattern in swing_path_patterns:
                if pattern in content:
                    violations.append({
                        'type': 'SWING_PATH_IN_CRYPTO',
                        'file': fpath.relative_to(WORKSPACE_ROOT),
                        'pattern': pattern,
                    })
                    print(f"✗ VIOLATION: {fpath.relative_to(WORKSPACE_ROOT)}")
                    print(f"    writes to swing path: {pattern}")
    except:
        pass

print(f"  → Found {len([v for v in violations if v['type']=='SWING_PATH_IN_CRYPTO'])} swing path violations")

# Summary
print(f"\n{'='*80}")
print(f"TOTAL VIOLATIONS: {len(violations)}")
if violations:
    print("VIOLATIONS FOUND - MUST FIX BEFORE PROCEEDING")
    for v in violations:
        print(f"  [{v['type']}] {v['file']}")
else:
    print("✓ NO CROSS-CONTAMINATION DETECTED")


## Section 3: Cross-Contamination Detection

Scan for violations:
- Crypto code importing from `core.strategies.equity.swing.*`
- Crypto code writing to swing directories
- Shared filenames between crypto and swing modules

In [ ]:
print("\n" + "="*80)
print("CRYPTO STRATEGY INVENTORY CHECK")
print("="*80)

REQUIRED_STRATEGIES = [
    'LongTermTrendFollowerStrategy',
    'VolatilityScaledSwingStrategy',
    'MeanReversionStrategy',
    'DefensiveHedgeShortStrategy',
    'CashStableAllocatorStrategy',
    'RecoveryReentryStrategy',
]

# Search for existing strategy implementations
found_strategies = {}
for root, dirs, files in os.walk(WORKSPACE_ROOT / 'crypto' / 'strategies'):
    for f in files:
        if f.endswith('.py') and f != '__init__.py':
            fpath = Path(root) / f
            try:
                with open(fpath) as fp:
                    content = fp.read()
                    for strategy_name in REQUIRED_STRATEGIES:
                        if f'class {strategy_name}' in content:
                            found_strategies[strategy_name] = fpath
                            print(f"✓ Found {strategy_name} in {fpath.relative_to(WORKSPACE_ROOT)}")
            except Exception as e:
                pass

# Check what's missing
missing = set(REQUIRED_STRATEGIES) - set(found_strategies.keys())
print(f"\n{'='*80}")
print(f"MISSING STRATEGIES: {len(missing)}")
print(f"{'='*80}")
for strategy in sorted(missing):
    print(f"✗ {strategy} - NEEDS IMPLEMENTATION")

print(f"\nFOUND STRATEGIES: {len(found_strategies)}/{len(REQUIRED_STRATEGIES)}")

# Also check for stub implementations (crypto_momentum, crypto_trend from earlier)
stub_strategies = list(Path(WORKSPACE_ROOT).glob('core/strategies/crypto/*.py'))
print(f"\nWARNING: Found stub strategies in core/strategies/crypto/ (should be removed/consolidated):")
for f in stub_strategies:
    if f.name != '__init__.py':
        print(f"  ⚠ {f.relative_to(WORKSPACE_ROOT)} - CONSOLIDATE/REMOVE")


## Section 2: Crypto Strategy Verification

All SIX crypto-only strategies must exist under `crypto/strategies/` namespace with:
- Separate files from swing strategies (NOT in `core/strategies/equity/swing/`)
- Required interface: `name`, `supported_regimes()`, `generate_signal(feature_context, portfolio_state, budget, regime_state)`
- No imports from swing strategy modules

**REQUIRED STRATEGIES:**
1. LongTermTrendFollowerStrategy - Follows long-term trends (supports RISK_ON, NEUTRAL)
2. VolatilityScaledSwingStrategy - Swing-like but with volatility scaling (supports NEUTRAL, RISK_ON)
3. MeanReversionStrategy - Reversion trades (ONLY NEUTRAL regime - gating enforced)
4. DefensiveHedgeShortStrategy - Short hedge (ONLY RISK_OFF, PANIC - time/size limited)
5. CashStableAllocatorStrategy - Cash preservation/allocation (all regimes)
6. RecoveryReentryStrategy - Reentry after crash (ONLY on PANIC→NEUTRAL transition)

In [ ]:
# STAGE 1 AUDIT: Market Data Ingestion
print("\n" + "="*80)
print("STAGE 1 AUDIT: Market Data Ingestion")
print("="*80)

audit_1 = {
    'stage': 'Market Data Ingestion',
    'expected_location': 'crypto/data/ or similar',
    'interface': 'fetch_crypto_ohlcv(universe, lookback_days) -> pd.DataFrame',
    'requirements': [
        'Pull OHLCV data for crypto universe in UTC',
        'Normalize symbols to internal canonical format',
        'Return DataFrame with OHLCV columns',
    ],
    'crypto_only_check': 'Must use crypto-specific data paths, NOT swing data paths',
}

crypto_data_files = list(Path(WORKSPACE_ROOT).glob('**/crypto/*data*.py'))
print(f"\n✓ Crypto data-related files found: {len(crypto_data_files)}")
for f in crypto_data_files[:5]:
    print(f"  - {f.relative_to(WORKSPACE_ROOT)}")

# Check if data fetcher exists
try:
    # Try to find market data ingestion module
    data_modules = list(Path(WORKSPACE_ROOT).glob('crypto/**/data*.py')) + \
                   list(Path(WORKSPACE_ROOT).glob('crypto/*universe*.py'))
    print(f"\n✓ Found universe/data modules: {len(data_modules)}")
    for m in data_modules:
        print(f"  - {m.relative_to(WORKSPACE_ROOT)}")
except Exception as e:
    print(f"✗ Error searching data modules: {e}")

audit_1['status'] = '⚠ CHECK: Need to verify data ingestion is crypto-only'
print(f"\nStage 1 Status: {audit_1['status']}")


## Section 1: Pipeline Architecture Audit

The mandatory 9-stage pipeline must execute in this exact order:
1. **Market Data Ingestion** - Pull/normalize OHLCV data
2. **Feature Builder** - Compute deterministic features (returns, ATR, vol, trend, correlation, drawdown)
3. **Regime Engine** - Output one regime enum (RISK_ON, NEUTRAL, RISK_OFF, PANIC)
4. **Strategy Selector** - Select max 2 active strategies with budget allocation
5. **Strategy Signals** - Each strategy generates Signal (LONG|SHORT|FLAT, size, confidence)
6. **Global Risk Manager** - Apply exposure/drawdown/correlation/overtrading caps + PANIC kill-switch
7. **Execution Engine** - Convert to orders, manage lifecycle (REUSE swing execution)
8. **Broker Adapter** - Kraken paper (simulated) + live (REST/WS)
9. **Reconciliation & Logging** - Immutable JSONL logs under crypto-only roots

In [ ]:
import os
import sys
from pathlib import Path
import json
import importlib
from typing import Dict, List, Set, Tuple

# Set up working directory
WORKSPACE_ROOT = Path('/Users/mohan/Documents/SandBox/test/trading_app')
os.chdir(WORKSPACE_ROOT)
sys.path.insert(0, str(WORKSPACE_ROOT))

print(f"✓ Working directory: {WORKSPACE_ROOT}")
print(f"✓ Python path configured")

# Define artifact roots
SWING_ROOTS = {
    'artifacts': '/data/artifacts/swing/',
    'logs': '/data/logs/swing/',
    'datasets': '/data/datasets/swing/',
    'ledger': '/data/ledger/swing/',
}

CRYPTO_ROOTS = {
    'artifacts': '/data/artifacts/crypto/kraken_global/',
    'logs': '/data/logs/crypto/kraken_global/',
    'datasets': '/data/datasets/crypto/kraken_global/',
    'ledger': '/data/ledger/crypto/kraken_global/',
}

# Verify roots are distinct
print("\n" + "="*80)
print("ARTIFACT PATH ISOLATION CHECK")
print("="*80)
for key in SWING_ROOTS:
    if SWING_ROOTS[key] == CRYPTO_ROOTS[key]:
        raise AssertionError(f"CRITICAL: {key} roots are identical! {SWING_ROOTS[key]}")
    print(f"✓ {key:12} isolated: swing={SWING_ROOTS[key]}, crypto={CRYPTO_ROOTS[key]}")


# Kraken Crypto Trading System: Comprehensive Audit & Implementation Guide

**Objective**: Senior engineering review of Kraken crypto implementation
- Verify pipeline order (9 stages in correct sequence)
- Implement ALL 6 crypto-only strategies with strict isolation
- Enforce artifact path separation (crypto vs swing)
- Add comprehensive test suite
- Cleanup temporary/debug files
- Generate audit report

**NO CROSS-CONTAMINATION RULE**: 
- Crypto must NOT import swing strategy classes
- Crypto artifacts must NOT write to swing directories
- Enforce with startup assertions + tests